In [139]:
import pandas as pd
import plotly.express as px
import re

In [140]:
csv_path = "./results_cellpdia30_sigma6_dilrad4_gliathr20.csv"

# Read the CSV file
df = pd.read_csv(csv_path)

# Convert the index into a column
df.reset_index(inplace=True)

# Extract 'tissue_location'
df['tissue_location'] = df['filename'].str.split('_40X_').str[-1]

# Extract 'staining_id'
df['staining_id'] = df['filename'].str.extract('(\d+)_40X')

# Display the first few rows of the DataFrame
df.head()


,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,%_dna_damage_signal,%_glia+_signal,tissue_location,staining_id
0,0,DSB Iba1 16_40X_CA1,0.568182,1.315789,0.840491,1.611765,57,75,132,163,1.096630,88.633728,CA1,16
1,1,DSB Iba1 16_40X_CA3,2.722581,2.992908,4.049080,4.230769,141,14,155,163,18.227673,96.349525,CA3,16
2,2,DSB Iba1 16_40X_CTX1,0.756757,1.647059,0.738739,1.822222,17,20,37,222,0.844288,13.598442,CTX1,16
3,3,DSB Iba1 16_40X_CTX2,1.748899,1.821101,2.535270,2.588983,218,9,227,241,39.405441,94.930744,CTX2,16
4,4,DSB Iba1 16_40X_CTX3,1.750000,1.866667,2.004785,2.277174,120,8,128,209,9.070396,54.768944,CTX3,16


In [141]:
mouse_id_csv_path = "./mouse_ids.csv"

# Read the CSV file
df_mouse_id = pd.read_csv(mouse_id_csv_path, delimiter=";", encoding="UTF-8")

# Display the first few rows of the DataFrame
df_mouse_id.head()

,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual
0,34,885,male,APP/PS1,poor
1,53,885,male,APP/PS1,good
2,23,2042,male,APP/PS1,good
3,31,2042,male,APP/PS1,good
4,43,2042,male,APP/PS1,good


In [142]:
df['staining_id'] = pd.to_numeric(df['staining_id'], errors='coerce')
df_mouse_id['staining_id'] = pd.to_numeric(df_mouse_id['staining_id'], errors='coerce')

# Merge both processed_results_df and mouse_id dataframes on staining_id
merged_df = pd.merge(df, df_mouse_id, on="staining_id")

# Display the first few rows of the DataFrame
merged_df.head()

,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,%_dna_damage_signal,%_glia+_signal,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual
0,0,DSB Iba1 16_40X_CA1,0.568182,1.315789,0.840491,1.611765,57,75,132,163,1.096630,88.633728,CA1,16,887,male,APP/PS1,poor
1,1,DSB Iba1 16_40X_CA3,2.722581,2.992908,4.049080,4.230769,141,14,155,163,18.227673,96.349525,CA3,16,887,male,APP/PS1,poor
2,2,DSB Iba1 16_40X_CTX1,0.756757,1.647059,0.738739,1.822222,17,20,37,222,0.844288,13.598442,CTX1,16,887,male,APP/PS1,poor
3,3,DSB Iba1 16_40X_CTX2,1.748899,1.821101,2.535270,2.588983,218,9,227,241,39.405441,94.930744,CTX2,16,887,male,APP/PS1,poor
4,4,DSB Iba1 16_40X_CTX3,1.750000,1.866667,2.004785,2.277174,120,8,128,209,9.070396,54.768944,CTX3,16,887,male,APP/PS1,poor


In [143]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_+_dna_damage_glia_nuclei',
                 hover_data=['staining_id','index'], title='Nr of DNA_damage+ glia+ nuclei by Tissue Location')

# Show the plot
fig.show()

In [144]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='avg_dna_damage_foci/glia_+',
                 hover_data=['staining_id'], title='DNA Damage in Glia Nuclei by Tissue Location')

# Show the plot
fig.show()

In [145]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei',
                 hover_data=['staining_id'], title='DNA Damage in All Nuclei by Tissue Location')

# Show the plot
fig.show()

In [146]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_glia_+_nuclei',
                 hover_data=['staining_id','index'], title='Nr of glia+ nuclei by Tissue Location')

# Show the plot
fig.show()

In [147]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_total_nuclei',
                 hover_data=['staining_id','index'], title='Nr of total nuclei by Tissue Location')

# Show the plot
fig.show()

In [148]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='nr_glia_+_nuclei',
                 hover_data=['tissue_location','index'], title='Nr of glia+ nuclei by Sample')

# Show the plot
fig.show()

In [149]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='%_dna_damage_signal',
                 hover_data=['tissue_location','index'], title='Dna damage mask area (QC)')

# Show the plot
fig.show()

In [150]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='%_glia+_signal',
                 hover_data=['tissue_location','index'], title='Glia mask area (QC)')

# Show the plot
fig.show()

We can observe there is a number of outliers in the glial and dna damage mask detection given the staining is suboptimal in some of the samples. I will filter the data to remove those suboptimal stains and just plot the optimal ones where the automated image analysis offers reliable results.

In [151]:
# Calculate mean area of the image occupied by glia+ signal
glia_mask_area_mean = df['%_glia+_signal'].mean() 

# Calculate mean area of the image occupied by dna_damage_+ signal
dna_damage_mask_area_mean = df['%_dna_damage_signal'].mean()  

In [152]:
# Define a function to determine staining quality
def determine_stain_quality(value, mean_value):
    if value < (mean_value + mean_value*0.55):
        return "optimal"
    else:
        return "suboptimal"

# Check stain quality for glia and create another column storing optimal or suboptimal if qc_passed or not    
merged_df['glia_stain_quality_auto'] = merged_df['%_glia+_signal'].apply(lambda x: determine_stain_quality(x, glia_mask_area_mean))

# Check stain quality for dna_damage and create another column storing optimal or suboptimal if qc_passed or not 
merged_df['dna_damage_stain_quality_auto'] = merged_df['%_dna_damage_signal'].apply(lambda x: determine_stain_quality(x, dna_damage_mask_area_mean))

# Check for both stain qualities and store True qc_passed if both are optimal
merged_df['staining_qc_passed'] = (merged_df['glia_stain_quality_auto'] == 'optimal') & (merged_df['dna_damage_stain_quality_auto'] == 'optimal')

# Group the DataFrame by 'staining_id' and check if all 'staining_qc_passed' values are True, otherwise set them all to False
merged_df['staining_qc_passed'] = merged_df.groupby('staining_id')['staining_qc_passed'].transform('all')

# Now, if all 'staining_qc_passed' values for the same 'staining_id' were True, the column will remain True; otherwise, it will be False

merged_df.head()

,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,...,%_glia+_signal,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,glia_stain_quality_auto,dna_damage_stain_quality_auto,staining_qc_passed
0,0,DSB Iba1 16_40X_CA1,0.568182,1.315789,0.840491,1.611765,57,75,132,163,...,88.633728,CA1,16,887,male,APP/PS1,poor,suboptimal,optimal,False
1,1,DSB Iba1 16_40X_CA3,2.722581,2.992908,4.049080,4.230769,141,14,155,163,...,96.349525,CA3,16,887,male,APP/PS1,poor,suboptimal,suboptimal,False
2,2,DSB Iba1 16_40X_CTX1,0.756757,1.647059,0.738739,1.822222,17,20,37,222,...,13.598442,CTX1,16,887,male,APP/PS1,poor,optimal,optimal,False
3,3,DSB Iba1 16_40X_CTX2,1.748899,1.821101,2.535270,2.588983,218,9,227,241,...,94.930744,CTX2,16,887,male,APP/PS1,poor,suboptimal,suboptimal,False
4,4,DSB Iba1 16_40X_CTX3,1.750000,1.866667,2.004785,2.277174,120,8,128,209,...,54.768944,CTX3,16,887,male,APP/PS1,poor,suboptimal,optimal,False


In [153]:
# Extract numbers from the CSV path
extracted_values = re.findall(r'\d+', csv_path)

# Dynamically assign the extracted values to variables
if len(extracted_values) >= 4:
    cellpose_nuclei_diameter = int(extracted_values[0])
    gaussian_sigma = int(extracted_values[1])
    dilation_radius_nuclei = int(extracted_values[2])
    glia_channel_threshold = int(extracted_values[3])

# Print the assigned analysis parameters
print(f"Cellpose nuclei diameter: {cellpose_nuclei_diameter}")
print(f"Gaussian sigma: {gaussian_sigma}")
print(f"Dilation radius nuclei: {dilation_radius_nuclei}")
print(f"Glia channel threshold: {glia_channel_threshold}")

Cellpose nuclei diameter: 30
Gaussian sigma: 6
Dilation radius nuclei: 4
Glia channel threshold: 20


In [154]:
# Save a copy of the qc_failed dataframe as .csv
merged_df.to_csv(f'qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_gliathr{glia_channel_threshold}.csv', index=False)

In [156]:
# Remove data from images with a poor quality stain (auto QC)
auto_filtered_df = merged_df[merged_df['staining_qc_passed'] == True]

In [158]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/glia_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Glia Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_gliathr{glia_channel_threshold}')

# Show the plot
fig.show()

In [160]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/glia_+_damage_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Damaged Glia Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_gliathr{glia_channel_threshold}')

# Show the plot
fig.show()

In [162]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in All Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_gliathr{glia_channel_threshold}')

# Show the plot
fig.show()

In [164]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei_damage_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Damage+ Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_gliathr{glia_channel_threshold}')

# Show the plot
fig.show()

Show failed qc dataframe

In [165]:
qc_failed_df = merged_df[merged_df['staining_qc_passed'] == False]

print(f"{qc_failed_df.shape[0]} stains have not passed QC and will need reanalysis")

qc_failed_list = qc_failed_df['index'].tolist()

qc_failed_df


72 stains have not passed QC and will need reanalysis


,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,...,%_glia+_signal,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,glia_stain_quality_auto,dna_damage_stain_quality_auto,staining_qc_passed
0,0,DSB Iba1 16_40X_CA1,0.568182,1.315789,0.840491,1.611765,57,75,132,163,...,88.633728,CA1,16,887,male,APP/PS1,poor,suboptimal,optimal,False
1,1,DSB Iba1 16_40X_CA3,2.722581,2.992908,4.049080,4.230769,141,14,155,163,...,96.349525,CA3,16,887,male,APP/PS1,poor,suboptimal,suboptimal,False
2,2,DSB Iba1 16_40X_CTX1,0.756757,1.647059,0.738739,1.822222,17,20,37,222,...,13.598442,CTX1,16,887,male,APP/PS1,poor,optimal,optimal,False
3,3,DSB Iba1 16_40X_CTX2,1.748899,1.821101,2.535270,2.588983,218,9,227,241,...,94.930744,CTX2,16,887,male,APP/PS1,poor,suboptimal,suboptimal,False
4,4,DSB Iba1 16_40X_CTX3,1.750000,1.866667,2.004785,2.277174,120,8,128,209,...,54.768944,CTX3,16,887,male,APP/PS1,poor,suboptimal,optimal,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,187,DSB Iba1 49_40X_CA3,0.529412,1.500000,1.990654,3.491803,6,11,17,107,...,6.575108,CA3,49,231,male,APP/PS1 x Neil3 KO,poor,optimal,optimal,False
188,188,DSB Iba1 49_40X_CTX1,0.944444,1.214286,1.448529,1.539062,14,4,18,136,...,8.087635,CTX1,49,231,male,APP/PS1 x Neil3 KO,poor,optimal,suboptimal,False
189,189,DSB Iba1 49_40X_CTX2,1.200000,1.500000,1.766871,1.945946,12,3,15,163,...,6.462860,CTX2,49,231,male,APP/PS1 x Neil3 KO,poor,optimal,suboptimal,False
190,190,DSB Iba1 49_40X_CTX3,0.500000,1.000000,1.229358,1.942029,5,5,10,109,...,2.494907,CTX3,49,231,male,APP/PS1 x Neil3 KO,poor,optimal,optimal,False
